In [ ]:
import cmdstanpy
from gptools.stan import compile_model
from gptools.stan.examples import sample_kwargs_from_env
from gptools.stan.examples.poisson_regression import simulate, plot_realization_1d
from gptools.util import Timer
from gptools.util.graph import lattice_predecessors, predecessors_to_edge_index
from gptools.util.kernels import ExpQuadKernel
from gptools.util.plotting import plot_band
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np

mpl.rcParams["figure.dpi"] = 144

We consider a simple Poisson regression model for count data $y$ with a latent log-rate $\eta$ subject to a Gaussian process prior. The model is specified by
$$\begin{align}
\eta &\sim \text{MultivariateNormal}\left(\mu, K\left(x,\alpha,\rho,\epsilon\right)\right)\\
y & \sim \text{Poisson}\left(\exp\eta\right),
\end{align}$$
where $x$ are the observation coordinates. We use a radial basis function kernel $K$ such that
$$
\text{cov}\left(\eta_i,\eta_j\right) = \alpha^2 \exp\left(-\frac{\left\vert x_i-x_j\right\vert^2}{2\rho^2}\right) + \epsilon\delta_{ij},
$$
where $\alpha$ is the covariance scale, $\rho$ is the correlation length, and $\epsilon$ is a diagonal noise term to ensure the covariance is positive-definite. A realization of the process is shown below. For simplicity, we assume that the kernel parameters are known in this example and use periodic boundary conditions.

In [ ]:
# Set up the parameters of the model and simulate forwards.
np.random.seed(1)
n = 100
kernel = ExpQuadKernel(1.2, 5, 1e-3, n)
mu = 1
x = np.arange(n)
realization = simulate(x, mu, kernel)
plot_realization_1d(realization).legend()

predecessors = lattice_predecessors(x.shape, 5)
edge_index = predecessors_to_edge_index(predecessors)

realization.update({
    "edge_index": edge_index,
    "num_edges": edge_index.shape[1],
})

# Naive Gaussian process implementation

We first fit the model using a centered parameterization, i.e., the parameters of the sampler directly correspond to the latent log-rate $\eta$. This parameterization is intuitive, but it can be inefficient when the data are not strongly informative. In this example, the information provided by the counts $y$ is relatively low because the mean rate is small. Consequently, $\eta$ is primarily constrained by the Gaussian process prior such that adjacent points are highly correlated (the data cannot decorrelated them). Posterior samples are thus highly correlated which slows down the sampler. But let's fit this model anyway.

In [ ]:
def sample_and_plot(stan_file: str, **kwargs) -> cmdstanpy.CmdStanMCMC:
    model = compile_model(stan_file=stan_file)
    with Timer(f"sampled using {stan_file}"):
        fit = model.sample(realization, **(sample_kwargs_from_env() | kwargs))
        
    ax = plot_realization_1d(realization)
    plot_band(x, np.exp(fit.stan_variable("eta")), label="inferred rate")
    ax.legend()
    return fit

In [ ]:
centered_fit = sample_and_plot("poisson_regression_centered.stan")

The model successfully infers the latent rate. However, in light of the weak data, we consider a non-centered parameterization of the model. In other words, we use a parameter $z\sim\text{Normal}\left(0, 1\right)$ and transform the white noise to a sample from the Gaussian process. This operation is analogous to sampling from a univariate Gaussian with mean $\mu$ and scale $\sigma$ by sampling from a standard Gaussian, multiplying by the scale $\sigma$, and adding the mean $\mu$. The two approaches are equivalent but the non-centered parameterization facilitates faster sampling for this example.

In [ ]:
non_centered_fit = sample_and_plot("poisson_regression_non_centered.stan")

Sampling succeeds as before, but the inference is faster because the white noise $z$ is uncorrelated under the prior and the weak data $y$ only induce minimal correlation. Let's consider the tree depth used by the sampler to explore the posterior (larger tree depths correspond to less efficient sampling).

In [ ]:
fig, ax = plt.subplots()
for key, fit in [("centered", centered_fit), ("non_centered", non_centered_fit)]:
    variables = fit.method_variables()
    treedepths = np.bincount(variables["treedepth__"].astype(int).ravel())
    ax.plot(treedepths, marker='o', label=key)
ax.legend()
ax.set_xlabel("Tree depth")
ax.set_ylabel("Frequency")

The majority of samples from the centered parameterization have tree depth greater than six. But all samples from the non-centered parametriazation have tree depth of six or less, explaining the difference in sampling efficiency.

# Graph likelihood

Considering only nearest neighbors in the precision matrix reduces the computational burden because we only need to invert many small matrices rather than one big one.

In [ ]:
sample_and_plot("poisson_regression_graph_centered.stan")

In [ ]:
sample_and_plot("poisson_regression_graph_non_centered.stan")

# Fourier likelihood

Employing the Fourier transform, we can evaluate the likelihood exactly without needing to invert the matrix if the grid is regular.

In [ ]:
sample_and_plot("poisson_regression_fourier_centered.stan")

In [ ]:
sample_and_plot("poisson_regression_fourier_non_centered.stan")